<a href="https://colab.research.google.com/github/AngeloMono/YOLACT-custom-weights/blob/main/YOLACT_custom_weights.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Google Colab Setup
Verify that the runtime type is set to GPU.
<br>
If running the following command return a string like:


> *NVIDIA-SMI has failed because it couldn't communicate with the NVIDIA driver. Make sure that the latest NVIDIA driver is installed and running.*

You need to change your runtime setup!
### Runtime Setup
- Go to Runtime > Change Runtime Type
- Choose GPU (TPU won't work)



In [ ]:
!nvidia-smi

Tue Jan 19 19:01:48 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   42C    P8     9W /  70W |      0MiB / 15079MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

# Load Dataset

## Create your custom dataset
Create in local yours custom dataset on your computer using the download of the project on the following link:
>https://github.com/wkentaro/labelme.git
>

Or using the following pip comand:
>*pip install labelme*
>
For more information on how to use this tool see the [README.md](https://github.com/wkentaro/labelme/blob/master/README.md) file in the project.

## Import the images and the label files

The following commands import from github the yolact project and create the Dataset folder inside it.

In [ ]:
# Clone the repo yolact
%cd /content/
!git clone "https://github.com/AngeloMono/yolact.git"

/content
Cloning into 'yolact'...
remote: Enumerating objects: 2936, done.
remote: Total 2936 (delta 0), reused 0 (delta 0), pack-reused 2936
Receiving objects: 100% (2936/2936), 21.21 MiB | 34.36 MiB/s, done.
Resolving deltas: 100% (1997/1997), done.


In [ ]:
# Make directories for dataset
%cd /content/yolact/
%mkdir Dataset
%mkdir "./Dataset/train_img/"
%mkdir "./Dataset/validation_img/"
%mkdir "./Dataset/coco_annotations/"

/content/yolact



Now that the folder that will contain the Dataset has been created, **you need to manually import** the images and the labels files into the folders *train_img* and *validation_img*.
<br>
<br>
Once the files have been imported we can continue with the conversion from the format provided by labelme into the coco notation using the **labelme2coco** lib.

In [ ]:
# Install labelme2coco
!pip install labelme2coco

In [ ]:
# Make sure we're in the train_img folder
%cd /content/yolact/Dataset/train_img/

# Create train_coco.json file
import labelme2coco
train_img_folder = "/content/yolact/Dataset/train_img"
save_coco_train_path = "./train_coco.json"
labelme2coco.convert(train_img_folder, save_coco_train_path)

# Move file train_coco.json into folder coco_annotations
!mv /content/yolact/Dataset/train_img/train_coco.json /content/yolact/Dataset/coco_annotations

/content/yolact/Dataset/train_img


In [ ]:
# Make sure we're in the validation_img folder
%cd /content/yolact/Dataset/validation_img/

# Create validation_coco.json file
test_img_folder = "/content/yolact/Dataset/validation_img"
save_coco_test_path = "./validation_coco.json"
labelme2coco.convert(test_img_folder, save_coco_test_path)

# Move file validation_coco.json into folder coco_annotations
!mv /content/yolact/Dataset/validation_img/validation_coco.json /content/yolact/Dataset/coco_annotations

/content/yolact/Dataset/validation_img


#Training the weight on the Custom dataset
##Download the weights that will be trained.
Download the weights  resnet50-19c8e357.pth from Google Drive using the download_google_drive.git project created by chentinghao.

In [ ]:
# Make sure we're in the top folder
%cd /content/

# Clone the repo download_google_drive
!git clone https://github.com/chentinghao/download_google_drive.git

/content
Cloning into 'download_google_drive'...
remote: Enumerating objects: 16, done.
remote: Total 16 (delta 0), reused 0 (delta 0), pack-reused 16
Unpacking objects: 100% (16/16), done.


In [ ]:
# Create a new directory for the weights
!mkdir -p /content/yolact/weights

# Download the weight to train
!python ./download_google_drive/download_gdrive.py 1Jy3yCdbatgXa5YYIdTCRrSV0S9V5g1rn ./yolact/weights/resnet50-19c8e357.pth

97.8MB [00:01, 67.4MB/s]


## Start the training

In [ ]:
# Start with training
%cd /content/yolact/
!python train.py --config=yolact_resnet50_cilia_config --batch_size=8

/content/yolact
loading annotations into memory...
Done (t=0.01s)
creating index...
index created!
loading annotations into memory...
Done (t=0.00s)
creating index...
index created!
/usr/local/lib/python3.6/dist-packages/torch/jit/_recursive.py:182: UserWarning: 'lat_layers' was found in ScriptModule constants,  but it is a non-constant submodule. Consider removing it.
  " but it is a non-constant {}. Consider removing it.".format(name, hint))
/usr/local/lib/python3.6/dist-packages/torch/jit/_recursive.py:182: UserWarning: 'pred_layers' was found in ScriptModule constants,  but it is a non-constant submodule. Consider removing it.
  " but it is a non-constant {}. Consider removing it.".format(name, hint))
/usr/local/lib/python3.6/dist-packages/torch/jit/_recursive.py:182: UserWarning: 'downsample_layers' was found in ScriptModule constants,  but it is a non-constant submodule. Consider removing it.
  " but it is a non-constant {}. Consider removing it.".format(name, hint))
Initializing

# Export Trained Weights and results
Now that you have trained your custom segmentation model, you can export the trained weights you have made here for inference on your device elsewhere

In [ ]:
%cd /content/
!zip -r results.zip '/content/yolact/logs/'
!zip -r results.zip '/content/yolact/weights'
!zip -d results.zip '/content/yolact/weights/resnet50-19c8e357.pth' 

/content
  adding: content/yolact/logs/ (stored 0%)
  adding: content/yolact/logs/yolact_resnet50_cilia.log (deflated 80%)
  adding: content/yolact/weights/ (stored 0%)
  adding: content/yolact/weights/yolact_resnet50_cilia_133_5614_interrupt.pth (deflated 7%)
  adding: content/yolact/weights/resnet50-19c8e357.pth (deflated 7%)
deleting: content/yolact/weights/resnet50-19c8e357.pth


## Congrats!


Congratulations, you can now download the results.zip file containing the trained weights and the results obtained.